<a href="https://colab.research.google.com/github/High-ren/Classification/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from glob import glob
from PIL import Image
import tensorflow_datasets as tfds
import tensorflow as tf

In [2]:
# Step 1: Load Dataset Automatically using TensorFlow Datasets
dataset, info = tfds.load('eurosat', split='train', with_info=True, as_supervised=True)

# Convert tf.data.Dataset to NumPy arrays (X, y)
X = []
y = []
for image, label in tfds.as_numpy(dataset):
    X.append(image)
    y.append(label)

X = np.array(X)
y = np.array(y)
classes = info.features['label'].names

print(f"Loaded {len(X)} images across {len(classes)} classes.")

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/eurosat/rgb/incomplete.7TTHU3_2.0.0/eurosat-train.tfrecord*...:   0%|     …

Dataset eurosat downloaded and prepared to /root/tensorflow_datasets/eurosat/rgb/2.0.0. Subsequent calls will reuse this data.
Loaded 27000 images across 10 classes.
